In [1]:
import numpy as np
import pandas as pd
import os
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader
from torch.autograd import Variable
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, f1_score, classification_report
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.neural_network import MLPClassifier
import xgboost as xgb
from scipy.stats import entropy
import matplotlib.pyplot as plt
import seaborn as sns
import warnings
import time
from tqdm import tqdm
from google.colab import drive

drive.mount('/content/drive/')
%cd /content/drive/MyDrive/Colab Notebooks/Katabatic/MedGAN/car/

# Suppress warnings
warnings.filterwarnings("ignore")

# Setting random seeds for reproducibility
torch.manual_seed(42)
np.random.seed(42)

# Check if CUDA is available
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
print(f"Using device: {device}")

Drive already mounted at /content/drive/; to attempt to forcibly remount, call drive.mount("/content/drive/", force_remount=True).
/content/drive/MyDrive/Colab Notebooks/Katabatic/MedGAN/car
Using device: cpu


In [2]:
def load_car_data(data_path):
    # Column names for the car dataset
    column_names = ['buying', 'maint', 'doors', 'persons', 'lug_boot', 'safety', 'class']

    # Load data
    df = pd.read_csv(data_path, header=None, names=column_names)

    print(f"Dataset shape: {df.shape}")

    # Split into train and test (80% train, 20% test)
    df_train, df_test = train_test_split(df, test_size=0.2, random_state=42)

    print(f"Training data shape: {df_train.shape}")
    print(f"Testing data shape: {df_test.shape}")

    # Split features and target
    X_train = df_train.drop('class', axis=1)
    y_train = df_train['class']
    X_test = df_test.drop('class', axis=1)
    y_test = df_test['class']

    # Identify categorical columns
    categorical_cols = X_train.columns.tolist()  # All columns are categorical

    # Create preprocessing pipeline for categorical data using one-hot encoding
    categorical_transformer = Pipeline(steps=[
        ('onehot', OneHotEncoder(handle_unknown='ignore', sparse_output=False))
    ])

    preprocessor = ColumnTransformer(
        transformers=[
            ('cat', categorical_transformer, categorical_cols)
        ])

    # Fit and transform the data
    X_train_transformed = preprocessor.fit_transform(X_train)
    X_test_transformed = preprocessor.transform(X_test)

    print(f"Processed training data shape: {X_train_transformed.shape}")
    print(f"Processed testing data shape: {X_test_transformed.shape}")

    # Get one-hot encoding feature names for later use
    cat_encoder = preprocessor.named_transformers_['cat'].named_steps['onehot']
    feature_names = cat_encoder.get_feature_names_out(categorical_cols)

    # Create label encoder for the target classes
    unique_classes = sorted(df['class'].unique())
    label_encoder = {cls: i for i, cls in enumerate(unique_classes)}
    inverse_label_encoder = {i: cls for cls, i in label_encoder.items()}

    # Encode targets
    y_train_encoded = y_train.map(label_encoder)
    y_test_encoded = y_test.map(label_encoder)

    # Store original categorical values
    cat_values = {}
    for col in categorical_cols:
        cat_values[col] = sorted(df[col].unique())

    return (X_train_transformed, y_train_encoded, X_test_transformed, y_test_encoded,
            preprocessor, feature_names, label_encoder, inverse_label_encoder, cat_values,
            X_train, y_train)

# Custom dataset class
class CarDataset(Dataset):
    def __init__(self, features, labels=None):
        self.features = torch.tensor(features, dtype=torch.float32)
        if labels is not None:
            self.labels = torch.tensor(labels.values, dtype=torch.long)
        else:
            self.labels = None

    def __len__(self):
        return len(self.features)

    def __getitem__(self, idx):
        if self.labels is not None:
            return self.features[idx], self.labels[idx]
        else:
            return self.features[idx]

# Autoencoder for MedGAN
class Autoencoder(nn.Module):
    def __init__(self, input_dim, hidden_dim=128, latent_dim=64):
        super(Autoencoder, self).__init__()

        # Encoder layers
        self.encoder = nn.Sequential(
            nn.Linear(input_dim, hidden_dim),
            nn.LeakyReLU(0.2),
            nn.BatchNorm1d(hidden_dim),

            nn.Linear(hidden_dim, latent_dim),
            nn.LeakyReLU(0.2),
            nn.BatchNorm1d(latent_dim)
        )

        # Decoder layers
        self.decoder = nn.Sequential(
            nn.Linear(latent_dim, hidden_dim),
            nn.LeakyReLU(0.2),
            nn.BatchNorm1d(hidden_dim),

            nn.Linear(hidden_dim, input_dim),
            nn.Sigmoid()  # Use sigmoid for binary features (one-hot encoded data)
        )

    def forward(self, x):
        encoded = self.encoder(x)
        decoded = self.decoder(encoded)
        return decoded

    def encode(self, x):
        return self.encoder(x)

# Generator Network for MedGAN
class Generator(nn.Module):
    def __init__(self, latent_dim, hidden_dim=128, output_dim=64):
        super(Generator, self).__init__()

        self.model = nn.Sequential(
            nn.Linear(latent_dim, hidden_dim),
            nn.LeakyReLU(0.2),
            nn.BatchNorm1d(hidden_dim),

            nn.Linear(hidden_dim, hidden_dim * 2),
            nn.LeakyReLU(0.2),
            nn.BatchNorm1d(hidden_dim * 2),

            nn.Linear(hidden_dim * 2, output_dim),
            nn.Tanh()  # Output is a latent representation for the decoder
        )

    def forward(self, z):
        return self.model(z)

# Discriminator Network for MedGAN
class Discriminator(nn.Module):
    def __init__(self, input_dim, hidden_dim=128):
        super(Discriminator, self).__init__()

        self.model = nn.Sequential(
            nn.Linear(input_dim, hidden_dim * 2),
            nn.LeakyReLU(0.2),
            nn.Dropout(0.3),

            nn.Linear(hidden_dim * 2, hidden_dim),
            nn.LeakyReLU(0.2),
            nn.Dropout(0.3),

            nn.Linear(hidden_dim, 1),
            nn.Sigmoid()  # Output probability of being real
        )

    def forward(self, x):
        return self.model(x)

# MedGAN Implementation
class MedGAN:
    def __init__(self, data_dim, latent_dim=100, hidden_dim=128, autoencoder_latent_dim=64):
        self.data_dim = data_dim
        self.latent_dim = latent_dim
        self.hidden_dim = hidden_dim
        self.autoencoder_latent_dim = autoencoder_latent_dim

        # Initialize networks
        self.autoencoder = Autoencoder(data_dim, hidden_dim, autoencoder_latent_dim).to(device)
        self.generator = Generator(latent_dim, hidden_dim, autoencoder_latent_dim).to(device)
        self.discriminator = Discriminator(data_dim, hidden_dim).to(device)

        # Setup optimizers
        self.ae_optimizer = optim.Adam(self.autoencoder.parameters(), lr=0.001)
        self.g_optimizer = optim.Adam(self.generator.parameters(), lr=0.0002, betas=(0.5, 0.999))
        self.d_optimizer = optim.Adam(self.discriminator.parameters(), lr=0.0002, betas=(0.5, 0.999))

        # Loss functions
        self.bce_loss = nn.BCELoss()
        self.mse_loss = nn.MSELoss()

        # Initialize loss tracking
        self.ae_losses = []
        self.g_losses = []
        self.d_losses = []

    def pretrain_autoencoder(self, data_loader, epochs, verbose=True):
        """Pretrain the autoencoder"""
        print("Pretraining autoencoder...")
        self.autoencoder.train()

        for epoch in range(epochs):
            epoch_loss = 0
            num_batches = 0

            for real_data, _ in data_loader:
                real_data = real_data.to(device)

                # Forward pass
                reconstructed = self.autoencoder(real_data)
                loss = self.mse_loss(reconstructed, real_data)

                # Backward pass and optimize
                self.ae_optimizer.zero_grad()
                loss.backward()
                self.ae_optimizer.step()

                epoch_loss += loss.item()
                num_batches += 1

            avg_loss = epoch_loss / num_batches
            self.ae_losses.append(avg_loss)

            if verbose and (epoch % 10 == 0 or epoch == epochs - 1):
                print(f"Autoencoder Epoch [{epoch+1}/{epochs}] | Loss: {avg_loss:.6f}")

    def train_gan(self, data_loader, epochs, d_steps=1, verbose=True):
      """Train the GAN after pretraining the autoencoder"""
      print("Training MedGAN...")
      self.autoencoder.eval()  # Freeze autoencoder weights

      for epoch in range(epochs):
          epoch_g_loss = 0
          epoch_d_loss = 0
          num_batches = 0

          for real_data, _ in data_loader:
              batch_size = real_data.size(0)  # Get the actual batch size
              real_data = real_data.to(device)

              # Create labels dynamically based on the actual batch size
              ones = torch.ones(batch_size, 1).to(device)
              zeros = torch.zeros(batch_size, 1).to(device)

              # Train Discriminator
              for _ in range(d_steps):
                  self.d_optimizer.zero_grad()

                  # Real data
                  d_real = self.discriminator(real_data)
                  d_real_loss = self.bce_loss(d_real, ones)

                  # Fake data
                  z = torch.randn(batch_size, self.latent_dim).to(device)
                  fake_latent = self.generator(z)
                  fake_data = self.autoencoder.decoder(fake_latent)
                  d_fake = self.discriminator(fake_data.detach())
                  d_fake_loss = self.bce_loss(d_fake, zeros)

                  # Total discriminator loss
                  d_loss = d_real_loss + d_fake_loss
                  d_loss.backward()
                  self.d_optimizer.step()

              # Train Generator
              self.g_optimizer.zero_grad()

              z = torch.randn(batch_size, self.latent_dim).to(device)
              fake_latent = self.generator(z)
              fake_data = self.autoencoder.decoder(fake_latent)
              g_fake = self.discriminator(fake_data)
              g_loss = self.bce_loss(g_fake, ones)

              g_loss.backward()
              self.g_optimizer.step()

              # Record losses
              epoch_d_loss += d_loss.item()
              epoch_g_loss += g_loss.item()
              num_batches += 1

          # Calculate average losses
          avg_d_loss = epoch_d_loss / num_batches
          avg_g_loss = epoch_g_loss / num_batches

          self.d_losses.append(avg_d_loss)
          self.g_losses.append(avg_g_loss)

          if verbose and (epoch % 10 == 0 or epoch == epochs - 1):
              print(f"GAN Epoch [{epoch+1}/{epochs}] | D Loss: {avg_d_loss:.6f} | G Loss: {avg_g_loss:.6f}")

    def generate_samples(self, num_samples):
        """Generate synthetic data samples"""
        self.generator.eval()
        self.autoencoder.eval()

        with torch.no_grad():
            z = torch.randn(num_samples, self.latent_dim).to(device)
            latent_codes = self.generator(z)
            samples = self.autoencoder.decoder(latent_codes).cpu().numpy()

        self.generator.train()
        self.autoencoder.train()

        return samples

    def save_model(self, path):
        """Save the model"""
        torch.save({
            'autoencoder_state_dict': self.autoencoder.state_dict(),
            'generator_state_dict': self.generator.state_dict(),
            'discriminator_state_dict': self.discriminator.state_dict(),
            'ae_optimizer_state_dict': self.ae_optimizer.state_dict(),
            'g_optimizer_state_dict': self.g_optimizer.state_dict(),
            'd_optimizer_state_dict': self.d_optimizer.state_dict(),
        }, path)

    def load_model(self, path):
        """Load the model"""
        checkpoint = torch.load(path)
        self.autoencoder.load_state_dict(checkpoint['autoencoder_state_dict'])
        self.generator.load_state_dict(checkpoint['generator_state_dict'])
        self.discriminator.load_state_dict(checkpoint['discriminator_state_dict'])
        self.ae_optimizer.load_state_dict(checkpoint['ae_optimizer_state_dict'])
        self.g_optimizer.load_state_dict(checkpoint['g_optimizer_state_dict'])
        self.d_optimizer.load_state_dict(checkpoint['d_optimizer_state_dict'])

# Post-process generated data for categorical features
def post_process_car_data(synthetic_data, preprocessor, cat_values, feature_names):
    """
    Post-process the synthetic data to convert one-hot encoded features back to categorical values
    """
    # Create a DataFrame with one-hot encoded columns
    synthetic_df = pd.DataFrame(synthetic_data, columns=feature_names)

    # Extract categorical feature groups
    result_df = pd.DataFrame()

    # Process each categorical column
    for col_name, values in cat_values.items():
        # Get one-hot columns for this feature
        col_pattern = f"{col_name}_"
        category_cols = [c for c in feature_names if c.startswith(col_pattern)]

        # Get the most likely category for each sample
        category_probs = synthetic_df[category_cols].values
        category_indices = np.argmax(category_probs, axis=1)

        # Map indices back to original categories
        # Extract the original category from the one-hot column name
        categories = [c.split('_', 1)[1] for c in category_cols]
        result_df[col_name] = [categories[idx] for idx in category_indices]

    return result_df


In [3]:
# Evaluation Metrics

# 1. Machine Learning Utility (TSTR)
def evaluate_tstr(real_data, synthetic_data, real_labels, random_state=42):
    """
    Train classifiers on synthetic data and test on real data (TSTR)
    Returns accuracy for each classifier
    """
    # Train-test split for real data
    X_train, X_test, y_train, y_test = train_test_split(
        real_data, real_labels, test_size=0.2, random_state=random_state
    )

    # Synthetic data (all used for training)
    X_synth = synthetic_data

    # Ensure proper dimensions for labels
    if isinstance(y_train, pd.Series):
        y_train = y_train.values

    # Create synthetic labels based on real distribution
    num_classes = len(np.unique(y_train))
    class_distribution = np.bincount(y_train.astype(int), minlength=num_classes) / len(y_train)
    np.random.seed(random_state)
    y_synth = np.random.choice(range(num_classes), size=len(X_synth), p=class_distribution)

    # Define classifiers
    classifiers = {
        'Logistic Regression': LogisticRegression(max_iter=1000, random_state=random_state),
        'MLP': MLPClassifier(hidden_layer_sizes=(100, 50), max_iter=500, random_state=random_state),
        'Random Forest': RandomForestClassifier(n_estimators=100, random_state=random_state),
        'XGBoost': xgb.XGBClassifier(n_estimators=100, random_state=random_state)
    }

    results = {}

    for name, clf in classifiers.items():
        # Train on synthetic data
        clf.fit(X_synth, y_synth)

        # Test on real data
        y_pred = clf.predict(X_test)

        # Calculate metrics
        accuracy = accuracy_score(y_test, y_pred)

        # We use weighted f1 since we have multiple classes
        f1 = f1_score(y_test, y_pred, average='weighted')

        results[name] = {
            'accuracy': accuracy,
            'f1_score': f1
        }

    return results

# 2. Statistical Similarity for categorical data
def evaluate_categorical_similarity(real_df, synthetic_df):
    """
    Calculate statistical similarity for categorical features
    """
    results = {'JSD': {}}

    # For each categorical column, calculate JSD
    for col in real_df.columns:
        # Get value counts
        real_counts = real_df[col].value_counts(normalize=True).sort_index()
        synth_counts = synthetic_df[col].value_counts(normalize=True).sort_index()

        # Align the distributions
        all_categories = sorted(set(real_counts.index) | set(synth_counts.index))
        real_dist = np.array([real_counts.get(cat, 0) for cat in all_categories])
        synth_dist = np.array([synth_counts.get(cat, 0) for cat in all_categories])

        # Add small epsilon to avoid zeros
        epsilon = 1e-10
        real_dist = real_dist + epsilon
        synth_dist = synth_dist + epsilon

        # Normalize
        real_dist = real_dist / real_dist.sum()
        synth_dist = synth_dist / synth_dist.sum()

        # Calculate JSD
        jsd = jensen_shannon_divergence(real_dist, synth_dist)
        results['JSD'][col] = jsd

    # Average JSD across all features
    results['JSD_avg'] = np.mean(list(results['JSD'].values()))

    return results

def jensen_shannon_divergence(p, q):
    """
    Calculate Jensen-Shannon Divergence between distributions p and q
    """
    # Ensure p and q are normalized
    p = p / np.sum(p)
    q = q / np.sum(q)

    m = 0.5 * (p + q)

    # Calculate JSD
    jsd = 0.5 * (entropy(p, m) + entropy(q, m))

    return jsd

def plot_loss_curves(model):
    """
    Plot the loss curves for the autoencoder, generator, and discriminator
    """
    # Plot autoencoder pretraining loss
    plt.figure(figsize=(12, 5))
    plt.subplot(1, 2, 1)
    plt.plot(model.ae_losses)
    plt.xlabel('Epoch')
    plt.ylabel('Loss')
    plt.title('Autoencoder Pretraining Loss')
    plt.grid(True)

    # Plot GAN losses
    plt.subplot(1, 2, 2)
    plt.plot(model.g_losses, label='Generator Loss')
    plt.plot(model.d_losses, label='Discriminator Loss')
    plt.xlabel('Epoch')
    plt.ylabel('Loss')
    plt.title('MedGAN Training Loss')
    plt.legend()
    plt.grid(True)

    plt.tight_layout()
    plt.savefig('car_medgan_loss_curves.png')
    plt.close()

def plot_categorical_distributions(real_df, synthetic_df):
    """
    Plot distributions of real vs synthetic data for categorical features
    """
    n_features = len(real_df.columns)

    plt.figure(figsize=(15, n_features * 4))

    for i, col in enumerate(real_df.columns):
        plt.subplot(n_features, 1, i+1)

        # Calculate proportions
        real_props = real_df[col].value_counts(normalize=True).sort_index()
        synth_props = synthetic_df[col].value_counts(normalize=True).sort_index()

        # Get all categories
        all_categories = sorted(set(real_props.index) | set(synth_props.index))

        # Create a DataFrame for plotting
        plot_df = pd.DataFrame({
            'Category': all_categories * 2,
            'Proportion': [real_props.get(cat, 0) for cat in all_categories] +
                         [synth_props.get(cat, 0) for cat in all_categories],
            'Type': ['Real'] * len(all_categories) + ['Synthetic'] * len(all_categories)
        })

        # Plot
        sns.barplot(x='Category', y='Proportion', hue='Type', data=plot_df)
        plt.title(f'Distribution for {col}')
        plt.xticks(rotation=45)
        plt.ylabel('Proportion')
        plt.legend()

    plt.tight_layout()
    plt.savefig('car_categorical_distributions.png')
    plt.close()

def plot_class_distribution(real_labels, synthetic_labels, label_encoder):
    """
    Plot the class distribution of real vs synthetic data
    """
    plt.figure(figsize=(12, 6))

    # Get class counts
    real_class_counts = pd.Series(real_labels).value_counts(normalize=True)
    synth_class_counts = pd.Series(synthetic_labels).value_counts(normalize=True)

    # Create inverse label encoder
    inverse_label_encoder = {v: k for k, v in label_encoder.items()}

    # Get all classes
    all_classes = sorted(set(real_class_counts.index) | set(synth_class_counts.index))

    # Create plot data
    plot_df = pd.DataFrame({
        'Class': [inverse_label_encoder.get(c, c) for c in all_classes] * 2,
        'Proportion': [real_class_counts.get(c, 0) for c in all_classes] +
                     [synth_class_counts.get(c, 0) for c in all_classes],
        'Type': ['Real'] * len(all_classes) + ['Synthetic'] * len(all_classes)
    })

    # Plot
    sns.barplot(x='Class', y='Proportion', hue='Type', data=plot_df)
    plt.title('Class Distribution: Real vs Synthetic Car Evaluations')
    plt.xticks(rotation=45)
    plt.tight_layout()
    plt.savefig('car_class_distribution.png')
    plt.close()

# Main function
def main():
    # File path
    data_path = 'data/car.csv'

    # Load and preprocess data
    (X_train_transformed, y_train, X_test_transformed, y_test,
     preprocessor, feature_names, label_encoder, inverse_label_encoder,
     cat_values, X_train_original, y_train_original) = load_car_data(data_path)

    # Create dataset and dataloader
    train_dataset = CarDataset(X_train_transformed, y_train)
    train_loader = DataLoader(train_dataset, batch_size=128, shuffle=True)

    # Initialize MedGAN model
    data_dim = X_train_transformed.shape[1]
    latent_dim = 100
    hidden_dim = 128
    autoencoder_latent_dim = 64

    print(f"Data dimension: {data_dim}")
    print(f"Latent dimension: {latent_dim}")
    print(f"Hidden dimension: {hidden_dim}")
    print(f"Autoencoder latent dimension: {autoencoder_latent_dim}")

    medgan = MedGAN(data_dim, latent_dim, hidden_dim, autoencoder_latent_dim)

    # Pretrain the autoencoder
    print("Pretraining the autoencoder...")
    ae_epochs = 100
    medgan.pretrain_autoencoder(train_loader, ae_epochs)

    # Train the GAN
    print("Training the GAN...")
    gan_epochs = 300
    medgan.train_gan(train_loader, gan_epochs)

    # Save the model
    print("Saving model...")
    medgan.save_model('car_medgan_model.pt')

    # Plot loss curves
    plot_loss_curves(medgan)

    # Generate synthetic data
    num_samples = 1000
    print(f"Generating {num_samples} synthetic samples...")
    synthetic_data_raw = medgan.generate_samples(num_samples)

    # Post-process the synthetic data
    synthetic_df = post_process_car_data(synthetic_data_raw, preprocessor, cat_values, feature_names)

    # Generate synthetic labels using a classifier trained on real data
    clf = RandomForestClassifier(n_estimators=100, random_state=42)
    clf.fit(X_train_transformed, y_train)
    synthetic_data_transformed = preprocessor.transform(synthetic_df)
    synthetic_labels_raw = clf.predict(synthetic_data_transformed)

    # Convert numeric labels to original class values
    synthetic_labels = [inverse_label_encoder[label] for label in synthetic_labels_raw]

    # Add class labels to the synthetic dataframe
    synthetic_df['class'] = synthetic_labels

    # Save the synthetic data
    synthetic_df.to_csv('synthetic_car_data_medgan.csv', index=False)

    # Plot distributions
    plot_categorical_distributions(X_train_original, synthetic_df.drop('class', axis=1))

    # Plot class distribution
    plot_class_distribution(y_train_original, synthetic_df['class'], label_encoder)

    # Statistical similarity evaluation
    print("Evaluating statistical similarity...")
    stat_results = evaluate_categorical_similarity(X_train_original, synthetic_df.drop('class', axis=1))

    print("\nJensen-Shannon Divergence (average):", stat_results['JSD_avg'])
    print("\nJSD per feature:")
    for feature, jsd in stat_results['JSD'].items():
        print(f"  {feature}: {jsd:.4f}")

    # Machine Learning Utility (TSTR) evaluation
    print("\nEvaluating Machine Learning Utility (TSTR)...")
    tstr_results = evaluate_tstr(X_train_transformed, synthetic_data_raw, y_train)

    print("\nTSTR Results:")
    for clf_name, metrics in tstr_results.items():
        print(f"{clf_name}: Accuracy = {metrics['accuracy']:.4f}, F1 Score = {metrics['f1_score']:.4f}")

    print("\nEvaluation complete! Check the output directory for plots and saved model.")


In [4]:
if __name__ == "__main__":
    main()

Dataset shape: (1729, 7)
Training data shape: (1383, 7)
Testing data shape: (346, 7)
Processed training data shape: (1383, 27)
Processed testing data shape: (346, 27)
Data dimension: 27
Latent dimension: 100
Hidden dimension: 128
Autoencoder latent dimension: 64
Pretraining the autoencoder...
Pretraining autoencoder...
Autoencoder Epoch [1/100] | Loss: 0.208357
Autoencoder Epoch [11/100] | Loss: 0.016645
Autoencoder Epoch [21/100] | Loss: 0.003024
Autoencoder Epoch [31/100] | Loss: 0.001315
Autoencoder Epoch [41/100] | Loss: 0.000752
Autoencoder Epoch [51/100] | Loss: 0.000522
Autoencoder Epoch [61/100] | Loss: 0.000381
Autoencoder Epoch [71/100] | Loss: 0.000307
Autoencoder Epoch [81/100] | Loss: 0.000262
Autoencoder Epoch [91/100] | Loss: 0.000210
Autoencoder Epoch [100/100] | Loss: 0.000185
Training the GAN...
Training MedGAN...
GAN Epoch [1/300] | D Loss: 1.366119 | G Loss: 0.670994
GAN Epoch [11/300] | D Loss: 1.205462 | G Loss: 0.787161
GAN Epoch [21/300] | D Loss: 0.950662 | G L